### Importing Libraries

In [9]:
import random #to create random integer for otp
import smtplib #to send mail using local smtp server
import re #to check email address is in correct format
from email.message import EmailMessage # convenient way to construct email messages with headers, content
import tkinter as tk #library used to build GUI
from tkinter import messagebox #creates a message window with an application-specified message, an icon and a set of buttons.
from datetime import datetime, timedelta # library used to check time email send and current time

### Function to Generate OTP

This function uses random module to generate a random integer between 100000 and 999999.





In [10]:
def generate_otp():
  return str(random.randint(100000, 999999))

### Function to Send OTP to user's email address

This function using local SMTP server sends the OTP to the user's email.

The function initially saves the sender email and app password instead of primary gmail password for security purpose.

Also a message with subject and content is set. HTML code is used to beautify the message content.

Using smtplib module a mail is send to the receiver email address.
Message submission is over TLS (transport layer security) protocol hence the port number is 465.

If any error occurs while sending, error message is prompted.

In [11]:
def send_otp_email(receiver_email, otp):
    sender_email = "nair.chandni.c@gmail.com"  # Placeholder sender email address
    pswd = "fpdf gmbn cknu gnyt"
    message = EmailMessage()
    message['Subject'] = 'OTP Verification'
    message['From'] = sender_email
    message['To'] = receiver_email
    html_content = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <link rel="stylesheet" type="text/css" hs-webfonts="true" href="https://fonts.googleapis.com/css?family=Lato|Lato:i,b,bi">
        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <style type="text/css">
            h1{{font-size:56px;}}
            h2{{font-size:28px;font-weight:900;}}
            p{{font-weight:100;}}
            td{{vertical-align:top;}}
            #email{{margin:auto;width:600px;background-color:#fff;}}
        </style>
    </head>
    <body bgcolor="#F5F8FA" style="width: 100%; font-family:Lato, sans-serif; font-size:18px;">
    <div id="email">
        <table role="presentation" width="100%">
            <tr>
                <td bgcolor="#00A4BD" align="center" style="color: white;">
                    <h4> Your Authentication Code is Ready!!</h4>
                </td>
            </tr>
        </table>
        <table role="presentation" border="0" cellpadding="0" cellspacing="10px" style="padding: 30px 30px 30px 60px;">
            <tr>
                <td>
                    <p>
                        Your OTP is: {otp} <br>
                        This OTP is valid for 5 mins.  <br>
                        Please do not share it with anyone. <br>

                        Thank You.
                    </p>
                </td>
            </tr>
        </table>
    </div>
    </body>
    </html>
    '''
    message.set_content(html_content, subtype='html')

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(sender_email, pswd)
            server.send_message(message)
    except Exception as e:
        print(f"Error sending email: {e}")


### Function to prompt the user to enter the OTP received in their email.

In [12]:
def enter_received_otp():
  return input("Enter the OTP received in your email: ")


### Function to verify the received OTP

This function return True if the entered otp and the generated otp mathces else False.

In [13]:
def verify_otp(entered_otp, generated_otp):
    return entered_otp == generated_otp

### Function to validate the email address

This function checks whether the entered email address of the receiver matches the regular expression pattern entirely. Returns True if it matches else False.

In [14]:
def email_validation(email):
  regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
  return re.fullmatch(regex, email) is not None

### Tkinter application class for OTP Verification

In [15]:
class OTPVerificationApp:
  def __init__(self, root):
    self.root = root
    self.root.title("OTP Verification App") #Title of the App

    self.label_email = tk.Label(root, text = "Enter your email address")
    self.label_email.grid(row = 0, column = 0, padx = 10, pady = 5)

    self.enter_email = tk.Entry(root)
    self.enter_email.grid(row=0, column=1, padx=10, pady = 5)

    self.send_button = tk.Button(root, text = "Send OTP", command=self.send_otp)
    self.send_button.grid(row=1,column=0, columnspan=2, padx=10,pady=5)

    self.label_otp = tk.Label(root, text = "Enter the OTP: ")
    self.label_otp.grid(row=2, column=0, padx=10,pady=5)

    self.enter_otp = tk.Entry(root)
    self.enter_otp.grid(row=2, column=1, padx=10, pady=5)

    self.verify_button = tk.Button(root, text = "Verify OTP", command=self.verify_otp)
    self.verify_button.grid(row=3, column=0, columnspan=2, padx=10,pady=5)

    self.attempts = 0  # Initialize attempts counter

  def send_otp(self):
        receiver_email = self.enter_email.get().strip()
        if email_validation(receiver_email):
            self.generated_otp = generate_otp()
            send_otp_email(receiver_email, self.generated_otp)
            self.otp_expiry_time = datetime.now() + timedelta(minutes=5)
            messagebox.showinfo("OTP Sent", "OTP has been sent to your email address.")
        else:
            messagebox.showerror("Invalid Email", "Please enter a valid email address.")

    # Function to verify OTP
  def verify_otp(self):
      if datetime.now() > self.otp_expiry_time:
            messagebox.showerror("OTP Expired", "The OTP has expired. Please request a new one.")
            return  
      entered_otp = self.enter_otp.get().strip()
      if verify_otp(entered_otp, self.generated_otp):
        messagebox.showinfo("Access Granted", "Access granted!")
        self.root.destroy()
        return
      else:
        self.attempts += 1  # Increment the attempts counter
        remaining_attempts = 3 - self.attempts
        if remaining_attempts > 0:
            messagebox.showerror("Incorrect OTP", f"Incorrect OTP. Attempts remaining: {remaining_attempts}")
        else:
            messagebox.showerror("Access Denied", "Too many incorrect attempts. Access denied.")
            self.root.destroy()
            return



In [16]:
def main():
  root = tk.Tk()
  app = OTPVerificationApp(root)
  root.mainloop()

if __name__ == "__main__":
    main()

